In [1]:
# this takes a minute or two since we are explaining over 30 thousand samples in a model with over a thousand trees
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

NameError: name 'shap' is not defined

In [2]:
shap.force_plot(explainer.expected_value, shap_values[0,:], processed.iloc[0,:-1])

NameError: name 'shap' is not defined

In [3]:
shap.force_plot(explainer.expected_value, shap_values[:1000,:], processed.iloc[:1000,:-1])

NameError: name 'shap' is not defined

In [ ]:
shap.summary_plot(shap_values, processed.iloc[:, :-1], plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X)